### This NoteBook provides some Notes about using PyTorch for Custom DataSet

In [20]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as f
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

import os

In [21]:
## config
device  = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Read and prepare the Data

In [22]:
## read the data
file_path = os.path.join(os.getcwd(), 'data', 'DryBeans', 'train.csv')

df_train = pd.read_csv(file_path)
df_train.head()

,ID,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,y
0,0,58238,971.303,397.202654,186.945510,2.124697,0.882317,58977,272.306674,0.604756,0.987470,0.775723,0.685561,0.006820,0.000929,0.469994,0.998595,HOROZ
1,1,44515,757.601,265.590470,213.967453,1.241266,0.592420,44780,238.071960,0.771745,0.994082,0.974620,0.896387,0.005966,0.002376,0.803510,0.997370,SEKER
2,2,31662,653.783,239.902428,168.421505,1.424417,0.712136,32034,200.781748,0.801407,0.988387,0.930853,0.836931,0.007577,0.002293,0.700453,0.997737,DERMASON
3,3,38778,734.020,268.446281,184.061923,1.458456,0.727925,39208,222.201897,0.766137,0.989033,0.904439,0.827733,0.006923,0.002005,0.685142,0.999250,DERMASON
4,4,42530,775.403,281.212824,193.236878,1.455275,0.726511,43028,232.703412,0.729816,0.988426,0.888895,0.827499,0.006612,0.001912,0.684755,0.996507,SIRA


In [4]:
## split the data
X = df_train.iloc[:, 1:-1]
y = df_train['y']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, 
                                                      shuffle=True, random_state=42)

# Convert Class String labels into Integers
encoder = LabelEncoder()
y_train_encoded = encoder.fit_transform(y_train)
y_valid_encoded = encoder.transform(y_valid)
## reshape them
y_train_encoded = y_train_encoded.reshape(-1, 1)
y_valid_encoded = y_valid_encoded.reshape(-1, 1)

# Normalize the input features of the dataset
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)

## convert to float32 then to tensors
X_train_scaled = torch.from_numpy(X_train_scaled.astype(np.float32))
X_valid_scaled = torch.from_numpy(X_valid_scaled.astype(np.float32))
y_train_encoded = torch.from_numpy(y_train_encoded)
y_valid_encoded = torch.from_numpy(y_valid_encoded)

#### Build a DataSet and DataLoader

In [5]:
learning_rate = 0.001
n_epochs = 50
n_batch_size = 32

In [6]:
## create two classes one for train and other for validation

class DryBeansTrainDataSet(Dataset):
    def __init__(self, transform=None):
        # get X (features) and y(taregt) ---> to tensors
        self.features = X_train_scaled
        self.target = y_train_encoded
        
        self.n_samples = X_train_scaled.shape[0]
        
    def __getitem__(self, index):
        ## data indexing 
        return self.features[index], torch.squeeze(self.target[index])
        
    def __len__(self):
        ## data length
        return self.n_samples
    
################################## ------------------ ##############################

class DryBeansValidDataSet(Dataset):
    def __init__(self, transform=None):
        # get X (features) and y(taregt) ---> to tensors
        self.features = X_valid_scaled
        self.target = y_valid_encoded
        
        self.n_samples = X_valid_scaled.shape[0]
        
    def __getitem__(self, index):
        ## data indexing 
        return self.features[index], torch.squeeze(self.target[index])
        
    def __len__(self):
        ## data length
        return self.n_samples

In [7]:
train_dataset = DryBeansTrainDataSet()
valid_dataset = DryBeansValidDataSet()

## create the dataloader
train_loader = DataLoader(dataset=train_dataset, batch_size=n_batch_size, 
                          shuffle=True, num_workers=0)
valid_loader = DataLoader(dataset=valid_dataset, batch_size=n_batch_size, 
                          shuffle=False, num_workers=0)

In [20]:
train_dataset[0]

(tensor([ 0.5609,  0.8261,  0.7590,  0.6468,  0.3316,  0.5259,  0.5645,  0.7519,
          0.3757, -0.5206, -0.8875, -0.4392, -0.9164, -0.8386, -0.4687, -0.2210]),
 tensor(0))

#### Build the Model

In [8]:
input_size = X_train_scaled.shape[1]
hidden_size = {'512':512, '256':256, '128':128, '64':64, '32':32}
num_classes = encoder.classes_.shape[0]

In [9]:
class FullyConnectedNetwork(nn.Module):
    def __init__(self, input_dim, n_classes, **hidden_dim):
        super(FullyConnectedNetwork, self).__init__()
        self.linear1 = nn.Linear(in_features=input_dim, out_features=hidden_dim['512'])
        self.linear2 = nn.Linear(in_features=hidden_dim['512'], out_features=hidden_dim['256'])
        self.linear3 = nn.Linear(in_features=hidden_dim['256'], out_features=hidden_dim['128'])
        self.linear4 = nn.Linear(in_features=hidden_dim['128'], out_features=hidden_dim['64'])
        self.linear5 = nn.Linear(in_features=hidden_dim['64'], out_features=hidden_dim['32'])
        self.linear6 = nn.Linear(in_features=hidden_dim['32'], out_features=n_classes)
        
    def forward(self, x):
        out = torch.tanh(self.linear1(x))
        out = f.dropout(out, 0.20)
        out = torch.tanh(self.linear2(out))
        out = f.dropout(out, 0.20)
        out = torch.tanh(self.linear3(out))
        out = f.dropout(out, 0.20)
        out = torch.tanh(self.linear4(out))
        out = f.dropout(out, 0.20)
        out = torch.tanh(self.linear5(out))
        out = f.dropout(out, 0.20)
        out = self.linear6(out)   ## No softmax at the end for multiclass --> crossEntropy handles
        return out

In [10]:
## Training
print('Training started _________________________________________ \n')

model = FullyConnectedNetwork(input_size, num_classes, **hidden_size).to(device=device)

## criteria
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

n_iterations = len(train_loader)     ## --> = len(train_dataset) / n_batch_size
n_train_samples = 0
n_train_correct = 0


for epoch in range(n_epochs):
    print(f'## epoch No. {epoch+1}')
    for i, (train_features, train_labels) in enumerate(train_loader):
        train_features = train_features.to(device=device)
        train_labels = train_labels.to(device=device)
        
        ## predicting then loss then backward then update weigths and empthy the gradeints
        y_pred_train = model(train_features)
        ## loss
        l = loss(y_pred_train, train_labels)
        ## backpropagation
        l.backward()
        ## optimization step
        optimizer.step()
        ## empty the gradients
        optimizer.zero_grad()

        ## during training
        ## get the max score --> return the value and index
        _, train_predictions = torch.max(y_pred_train, dim=1)
        
        n_train_samples += train_labels.shape[0]
        n_train_correct += (train_predictions==train_labels).sum().item()
        train_acc = (n_train_correct/n_train_samples)
        print(f'Iteration No. {i+1} | loss={l:.5f} | acc={train_acc:.5f}')
        
        
print('Training finished ___________________________________________ \n')   

Training started _________________________________________ 

## epoch No. 1
Iteration No. 1 | loss=1.95178 | acc=0.21875
Iteration No. 2 | loss=1.86579 | acc=0.35938
Iteration No. 3 | loss=1.81087 | acc=0.41667
Iteration No. 4 | loss=1.67832 | acc=0.48438
Iteration No. 5 | loss=1.65055 | acc=0.53125
Iteration No. 6 | loss=1.58931 | acc=0.55208
Iteration No. 7 | loss=1.48572 | acc=0.56696
Iteration No. 8 | loss=1.52877 | acc=0.57422
Iteration No. 9 | loss=1.51610 | acc=0.55903
Iteration No. 10 | loss=1.38302 | acc=0.56250
Iteration No. 11 | loss=1.51986 | acc=0.55398
Iteration No. 12 | loss=1.31938 | acc=0.56510
Iteration No. 13 | loss=1.55386 | acc=0.54567
Iteration No. 14 | loss=1.34245 | acc=0.54911
Iteration No. 15 | loss=1.30904 | acc=0.55417
Iteration No. 16 | loss=1.26067 | acc=0.56055
Iteration No. 17 | loss=1.13584 | acc=0.57537
Iteration No. 18 | loss=1.30098 | acc=0.57639
Iteration No. 19 | loss=1.25687 | acc=0.57072
Iteration No. 20 | loss=1.28783 | acc=0.57031
Iteration No.

Iteration No. 179 | loss=0.28606 | acc=0.80744
Iteration No. 180 | loss=0.44235 | acc=0.80764
Iteration No. 181 | loss=0.44837 | acc=0.80818
Iteration No. 182 | loss=0.62110 | acc=0.80804
Iteration No. 183 | loss=0.22415 | acc=0.80908
Iteration No. 184 | loss=0.27213 | acc=0.80961
Iteration No. 185 | loss=0.31662 | acc=0.81030
Iteration No. 186 | loss=0.25739 | acc=0.81099
Iteration No. 187 | loss=0.42123 | acc=0.81116
Iteration No. 188 | loss=0.44305 | acc=0.81134
Iteration No. 189 | loss=0.22977 | acc=0.81217
Iteration No. 190 | loss=0.51668 | acc=0.81234
Iteration No. 191 | loss=0.43237 | acc=0.81250
Iteration No. 192 | loss=0.49244 | acc=0.81283
Iteration No. 193 | loss=0.23036 | acc=0.81347
Iteration No. 194 | loss=0.38530 | acc=0.81379
Iteration No. 195 | loss=0.55345 | acc=0.81394
Iteration No. 196 | loss=0.42932 | acc=0.81425
Iteration No. 197 | loss=0.35123 | acc=0.81456
Iteration No. 198 | loss=0.59104 | acc=0.81455
Iteration No. 199 | loss=0.24919 | acc=0.81533
Iteration No.

Iteration No. 98 | loss=0.21847 | acc=0.85665
Iteration No. 99 | loss=0.22626 | acc=0.85695
Iteration No. 100 | loss=0.42442 | acc=0.85700
Iteration No. 101 | loss=0.25240 | acc=0.85721
Iteration No. 102 | loss=0.30944 | acc=0.85735
Iteration No. 103 | loss=0.18230 | acc=0.85764
Iteration No. 104 | loss=0.22233 | acc=0.85786
Iteration No. 105 | loss=0.26966 | acc=0.85799
Iteration No. 106 | loss=0.22820 | acc=0.85828
Iteration No. 107 | loss=0.21069 | acc=0.85849
Iteration No. 108 | loss=0.19018 | acc=0.85870
Iteration No. 109 | loss=0.24703 | acc=0.85899
Iteration No. 110 | loss=0.35460 | acc=0.85911
Iteration No. 111 | loss=0.39914 | acc=0.85915
Iteration No. 112 | loss=0.18901 | acc=0.85944
Iteration No. 113 | loss=0.37542 | acc=0.85956
Iteration No. 114 | loss=0.31675 | acc=0.85968
Iteration No. 115 | loss=0.21396 | acc=0.85988
Iteration No. 116 | loss=0.32905 | acc=0.86000
Iteration No. 117 | loss=0.50376 | acc=0.85988
Iteration No. 118 | loss=0.14644 | acc=0.86016
Iteration No. 1

Iteration No. 35 | loss=0.22898 | acc=0.87667
Iteration No. 36 | loss=0.32406 | acc=0.87677
Iteration No. 37 | loss=0.23371 | acc=0.87693
Iteration No. 38 | loss=0.34764 | acc=0.87704
Iteration No. 39 | loss=0.22025 | acc=0.87709
Iteration No. 40 | loss=0.41449 | acc=0.87708
Iteration No. 41 | loss=0.37813 | acc=0.87713
Iteration No. 42 | loss=0.21131 | acc=0.87718
Iteration No. 43 | loss=0.22755 | acc=0.87728
Iteration No. 44 | loss=0.58563 | acc=0.87701
Iteration No. 45 | loss=0.20564 | acc=0.87706
Iteration No. 46 | loss=0.26925 | acc=0.87711
Iteration No. 47 | loss=0.25158 | acc=0.87722
Iteration No. 48 | loss=0.06054 | acc=0.87742
Iteration No. 49 | loss=0.18153 | acc=0.87758
Iteration No. 50 | loss=0.60670 | acc=0.87747
Iteration No. 51 | loss=0.08952 | acc=0.87768
Iteration No. 52 | loss=0.37602 | acc=0.87772
Iteration No. 53 | loss=0.15506 | acc=0.87788
Iteration No. 54 | loss=0.53796 | acc=0.87782
Iteration No. 55 | loss=0.39657 | acc=0.87787
Iteration No. 56 | loss=0.19973 | 

Iteration No. 211 | loss=0.22339 | acc=0.88462
Iteration No. 212 | loss=0.24078 | acc=0.88473
Iteration No. 213 | loss=0.41111 | acc=0.88468
Iteration No. 214 | loss=0.34396 | acc=0.88467
Iteration No. 215 | loss=0.22268 | acc=0.88478
Iteration No. 216 | loss=0.07498 | acc=0.88493
Iteration No. 217 | loss=0.19294 | acc=0.88504
Iteration No. 218 | loss=0.15090 | acc=0.88519
Iteration No. 219 | loss=0.53087 | acc=0.88514
Iteration No. 220 | loss=0.27519 | acc=0.88516
Iteration No. 221 | loss=0.15569 | acc=0.88519
Iteration No. 222 | loss=0.12985 | acc=0.88530
Iteration No. 223 | loss=0.19745 | acc=0.88541
Iteration No. 224 | loss=0.44838 | acc=0.88540
Iteration No. 225 | loss=0.29113 | acc=0.88542
Iteration No. 226 | loss=0.15423 | acc=0.88549
Iteration No. 227 | loss=0.25405 | acc=0.88556
Iteration No. 228 | loss=0.10597 | acc=0.88567
Iteration No. 229 | loss=0.32686 | acc=0.88565
Iteration No. 230 | loss=0.31296 | acc=0.88568
Iteration No. 231 | loss=0.17445 | acc=0.88571
Iteration No.

...

Iteration No. 9 | loss=0.43040 | acc=0.92011
Iteration No. 10 | loss=0.09657 | acc=0.92012
Iteration No. 11 | loss=0.20861 | acc=0.92011
Iteration No. 12 | loss=0.20670 | acc=0.92011
Iteration No. 13 | loss=0.11864 | acc=0.92011
Iteration No. 14 | loss=0.17623 | acc=0.92011
Iteration No. 15 | loss=0.21217 | acc=0.92011
Iteration No. 16 | loss=0.09746 | acc=0.92011
Iteration No. 17 | loss=0.29571 | acc=0.92011
Iteration No. 18 | loss=0.08935 | acc=0.92012
Iteration No. 19 | loss=0.36098 | acc=0.92011
Iteration No. 20 | loss=0.07231 | acc=0.92012
Iteration No. 21 | loss=0.24627 | acc=0.92011
Iteration No. 22 | loss=0.38302 | acc=0.92011
Iteration No. 23 | loss=0.16680 | acc=0.92011
Iteration No. 24 | loss=0.14964 | acc=0.92011
Iteration No. 25 | loss=0.16440 | acc=0.92011
Iteration No. 26 | loss=0.18107 | acc=0.92011
Iteration No. 27 | loss=0.27467 | acc=0.92011
Iteration No. 28 | loss=0.58663 | acc=0.92010
Iteration No. 29 | loss=0.15607 | acc=0.92010
Iteration No. 30 | loss=0.07983 | a

Iteration No. 229 | loss=0.07853 | acc=0.92026
Iteration No. 230 | loss=0.09072 | acc=0.92026
Iteration No. 231 | loss=0.17683 | acc=0.92026
Iteration No. 232 | loss=0.30011 | acc=0.92026
Iteration No. 233 | loss=0.22575 | acc=0.92026
Iteration No. 234 | loss=0.14514 | acc=0.92026
Iteration No. 235 | loss=0.30256 | acc=0.92026
Iteration No. 236 | loss=0.13491 | acc=0.92026
Iteration No. 237 | loss=0.35075 | acc=0.92025
Iteration No. 238 | loss=0.20473 | acc=0.92025
Iteration No. 239 | loss=0.12235 | acc=0.92025
Iteration No. 240 | loss=0.11185 | acc=0.92026
Iteration No. 241 | loss=0.16063 | acc=0.92026
Iteration No. 242 | loss=0.03291 | acc=0.92026
Iteration No. 243 | loss=0.33877 | acc=0.92026
Iteration No. 244 | loss=0.37216 | acc=0.92026
Iteration No. 245 | loss=0.39649 | acc=0.92025
Iteration No. 246 | loss=0.19191 | acc=0.92025
Iteration No. 247 | loss=0.16865 | acc=0.92025
Iteration No. 248 | loss=0.11218 | acc=0.92026
Iteration No. 249 | loss=0.34366 | acc=0.92025
Iteration No.

Iteration No. 179 | loss=0.07899 | acc=0.92035
Iteration No. 180 | loss=0.12172 | acc=0.92034
Iteration No. 181 | loss=0.43849 | acc=0.92034
Iteration No. 182 | loss=0.17843 | acc=0.92034
Iteration No. 183 | loss=0.13731 | acc=0.92034
Iteration No. 184 | loss=0.13817 | acc=0.92034
Iteration No. 185 | loss=0.13772 | acc=0.92034
Iteration No. 186 | loss=0.29382 | acc=0.92034
Iteration No. 187 | loss=0.36583 | acc=0.92034
Iteration No. 188 | loss=0.08427 | acc=0.92034
Iteration No. 189 | loss=0.10538 | acc=0.92034
Iteration No. 190 | loss=0.13398 | acc=0.92035
Iteration No. 191 | loss=0.28256 | acc=0.92035
Iteration No. 192 | loss=0.07010 | acc=0.92035
Iteration No. 193 | loss=0.18728 | acc=0.92035
Iteration No. 194 | loss=0.07950 | acc=0.92036
Iteration No. 195 | loss=0.15074 | acc=0.92036
Iteration No. 196 | loss=0.09900 | acc=0.92036
Iteration No. 197 | loss=0.25437 | acc=0.92036
Iteration No. 198 | loss=0.17716 | acc=0.92036
Iteration No. 199 | loss=0.16005 | acc=0.92036
Iteration No.

Iteration No. 85 | loss=0.11152 | acc=0.92048
Iteration No. 86 | loss=0.17729 | acc=0.92048
Iteration No. 87 | loss=0.12634 | acc=0.92049
Iteration No. 88 | loss=0.29317 | acc=0.92048
Iteration No. 89 | loss=0.06029 | acc=0.92049
Iteration No. 90 | loss=0.16362 | acc=0.92049
Iteration No. 91 | loss=0.12133 | acc=0.92049
Iteration No. 92 | loss=0.34476 | acc=0.92050
Iteration No. 93 | loss=0.16583 | acc=0.92050
Iteration No. 94 | loss=0.25670 | acc=0.92049
Iteration No. 95 | loss=0.12504 | acc=0.92050
Iteration No. 96 | loss=0.07254 | acc=0.92050
Iteration No. 97 | loss=0.19698 | acc=0.92050
Iteration No. 98 | loss=0.31793 | acc=0.92050
Iteration No. 99 | loss=0.21438 | acc=0.92050
Iteration No. 100 | loss=0.06046 | acc=0.92050
Iteration No. 101 | loss=0.10824 | acc=0.92051
Iteration No. 102 | loss=0.13928 | acc=0.92051
Iteration No. 103 | loss=0.37296 | acc=0.92050
Iteration No. 104 | loss=0.04895 | acc=0.92051
Iteration No. 105 | loss=0.10464 | acc=0.92051
Iteration No. 106 | loss=0.2

Iteration No. 269 | loss=0.23584 | acc=0.92066
Iteration No. 270 | loss=0.21125 | acc=0.92066
Iteration No. 271 | loss=0.12080 | acc=0.92066
Training finished ___________________________________________ 



#### Evaluation

In [11]:
## when you evaluate you must stop gradients to not kick the requires_grad=True
n_valid_correct = 0
n_valid_samples = 0

with torch.no_grad():
    for i, (valid_features, valid_labels) in enumerate(valid_loader):
        valid_features = valid_features.to(device=device)
        valid_labels = valid_labels.to(device=device)

        y_pred_valid = model(valid_features)

        ## get the max score --> return the value and index
        _, valid_predictions = torch.max(y_pred_valid, dim=1)
        
        n_valid_samples += valid_labels.shape[0]
        n_valid_correct += (valid_predictions==valid_labels).sum().item()
        
        
valid_acc = (n_valid_correct/n_valid_samples) * 100.0
print(f'Validation Accuracy is .. {valid_acc} %')

Validation Accuracy is .. 91.97046608214121 %


### Done !